In [1]:
import open3d as o3d
import numpy as np
from glob import glob 
np.set_printoptions(threshold=np.inf)

print("Load a ply point cloud, print it, and render it")
dir = '/media/an/New Volume/cleargrasp/results/new/bae_pidi_mkdc/exp-000/output-point-cloud'
path = sorted(glob(f"{dir}/*-pointcloud.ply"))

for i in range(len(path)):
    pcd = o3d.io.read_point_cloud(path[i])
    R = pcd.get_rotation_matrix_from_xyz((np.pi, 0, 0))
    pcd.rotate(R, center= pcd.get_center())
    o3d.visualization.draw_geometries([pcd])


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Load a ply point cloud, print it, and render it
